In [1]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/')

In [2]:
import buffalo.ingestion

In [3]:
buffalo.ingestion.Configuration.print()

{'storage_folder': '../__cached_data', 'api_keys': {<API.ADVANTAGE: 1>: '2rrNROO0beX90lPH7ixQOp0mT_9SwF0d'}, 'watched_tickers': ['IBM', 'MSFT']}


In [6]:
import sqlite3


fp = r'~/newsqlitefile.sqlite'

import pandas as pd

a = pd.DataFrame({"a": range(10), "v": range(1, 11)})

newconn, _ = sqlite3.connect(fp)
a.to_sql("a", newconn)

newconn.close()

OperationalError: unable to open database file